In [ ]:
#output csv with title
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import time

def createDataFrame(t):
    #put data into a list
    title_list = [p.text.strip() for p in t]
    
    global df
    #check if dataframe exist
    if c == 0:
        df = pd.DataFrame({'title': title_list})
    else:
        df = df.append(pd.DataFrame({'title': title_list}), ignore_index = True)
    
    #check fake
    cond1 = df['title'].str.contains('【錯誤】')
    cond2 = df['title'].str.contains('【部分錯誤】')
    '''
    if title contains 【錯誤】:
        put 'fake'
    elif contains 【部分錯誤】:
        put 'partial error'
    else:
        put 'other'
    '''
    df['tag'] = np.where(cond1, '__label__fake', np.where(cond2, '__label__partialError', '__label__other'))
    df = df[['tag','title']]
    
def getData(url):
    #set user agent
    req = request.Request(url)
    req.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36")
    response = request.urlopen(req)
    data = response.read().decode("utf-8")

    #read data in html format
    html = BeautifulSoup(data,"html.parser")
    
    #set find area
    t = html.find_all("h3", class_= "article-title")
    
    #create dataframe
    createDataFrame(t) 
    
    #find next page url
    nextpage = html.find("a", string = "下一頁 ›")
    return nextpage["href"]
    time.sleep(5)

def outputCsv(): 
    #input news article URL
    #錯誤https://tfc-taiwan.org.tw/articles/category/26/27
    #部分錯誤https://tfc-taiwan.org.tw/articles/category/26/28
    fakeURL = "https://tfc-taiwan.org.tw/articles/category/26/27"
    partial_errorURL = "https://tfc-taiwan.org.tw/articles/category/26/28"
    global c
    c = 0
    for i in range(5):
        #get next page URL from getData function
        fakeURL = "https://tfc-taiwan.org.tw" + getData(fakeURL)   
        c+=1
        time.sleep(3)
    
    for i in range(5):
        partial_errorURL = "https://tfc-taiwan.org.tw" + getData(partial_errorURL)
        time.sleep(3)
    
    df['title'] = df['title'].str.replace('【錯誤】','').str.replace('【部分錯誤】','').str.replace('網傳','')
    pd.set_option('max_colwidth',100)    
    display(df)
    
    #output csv file
    path = "C:\\Users\\user\Desktop\\newscrawler\\outputcsv\\csvdata\\"
    df.to_csv(path + 'data.csv', encoding='utf_8_sig', index=False, header=False)
    
def sleeptime(h,m,s):
    return h*3600 + m*60 + s

#update every 6 hours
second = sleeptime(6,0,0)

if __name__ == '__main__':
    while True:  
        outputCsv()
        time.sleep(second)


,tag,title
0,__label__fake,"影片宣稱「影片中的年輕人,就是LWKS患者...正常人不會帶著小刀去附近商店買東西...現今隨機殺人or傷人案件頻傳，大家要小心...」？"
1,__label__fake,「以色列衛生部長稱新冠肺炎是上天對同性戀的懲罰，隨後被確診感染」、「以色列衛生部長稱只有同性戀才得武肺 自己被確診」？
2,__label__fake,「日本的諾貝爾醫學獎得主教授本庶佑（Tasuku Honjo）博士今天說，冠狀病毒不是天然的...我在中國武漢實驗室工作了4年...」？
3,__label__fake,影片宣稱「絕密視頻流出，時間不詳 染疫死的？處理屍體的軍人沒穿防護服...」、「中共軍人掩埋大批屍體視頻曝光」？
4,__label__fake,影片宣稱「中國5G技術，應用在演唱會上技驚四座，難怪美國人要打壓華為」？
...,...,...
95,__label__partialError,「吃冷的米飯能抗腸癌」？
96,__label__partialError,媒體報導「研究證實吃辣可能導致失智症...研究結果顯示，每天吃超過50g辣椒的人，記憶力衰退和認知能力變差的風險為一般人的2倍」？
97,__label__partialError,「家禽之王梵天雞 身高超過1m...體型如五六歲小孩」？
98,__label__partialError,「高速公路所有／十八處交流道都裝設攝影機，若違規插隊、壓到雙白線或未打方向燈都會被開罰單」？
